<a href="https://colab.research.google.com/github/talkin24/GradientBoosting/blob/main/Ch10_deploy_XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/GradientBoosting

Mounted at /content/drive
/content/drive/MyDrive/GradientBoosting


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('./student-por.csv')
df.head()

,school;sex;age;address;famsize;Pstatus;Medu;Fedu;Mjob;Fjob;reason;guardian;traveltime;studytime;failures;schoolsup;famsup;paid;activities;nursery;higher;internet;romantic;famrel;freetime;goout;Dalc;Walc;health;absences;G1;G2;G3
0,"GP;NaN;18;""U"";""GT3"";""A"";4;4;""at_home"";""teacher..."
1,"GP;""F"";NaN;""U"";""GT3"";""T"";1;1;""at_home"";""other""..."
2,"GP;""F"";15;""U"";""LE3"";""T"";1;1;""at_home"";""other"";..."
3,"GP;""F"";15;""U"";""GT3"";""T"";4;2;""health"";""services..."
4,"GP;""F"";16;""U"";""GT3"";""T"";3;3;""other"";""other"";""h..."


In [4]:
df = pd.read_csv('./student-por.csv', sep=';')

In [5]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,NaN,18.0,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,NaN,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15.0,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15.0,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16.0,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [6]:
df.isnull().sum()

school        0
sex           1
age           1
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      1
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
G3            0
dtype: int64

In [7]:
df[df.isna().any(axis=1)]

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,NaN,18.0,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,NaN,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11


In [8]:
pd.options.display.max_columns = None

In [9]:
df[df.isna().any(axis=1)]

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,NaN,18.0,U,GT3,A,4,4,at_home,teacher,course,NaN,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,NaN,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11


In [10]:
df['age'] = df['age'].fillna(-999.0)

In [11]:
df['sex'] = df['sex'].fillna(df['sex'].mode())
df['guardian'] = df['guardian'].fillna(df['guardian'].mode())

In [12]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18.0,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,-999.0,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15.0,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15.0,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16.0,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


In [13]:
categorical_columns = df.columns[df.dtypes==object].tolist()

In [14]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

In [15]:
hot = ohe.fit_transform(df[categorical_columns])

In [16]:
hot_df = pd.DataFrame(hot.toarray())
hot_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0


In [17]:
print(hot)

  (0, 0)	1.0
  (0, 2)	1.0
  (0, 5)	1.0
  (0, 6)	1.0
  (0, 8)	1.0
  (0, 10)	1.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 25)	1.0
  (0, 28)	1.0
  (0, 29)	1.0
  (0, 31)	1.0
  (0, 33)	1.0
  (0, 36)	1.0
  (0, 38)	1.0
  (0, 39)	1.0
  (0, 41)	1.0
  (1, 0)	1.0
  (1, 2)	1.0
  (1, 5)	1.0
  (1, 6)	1.0
  (1, 9)	1.0
  (1, 10)	1.0
  (1, 17)	1.0
  (1, 20)	1.0
  :	:
  (647, 27)	1.0
  (647, 29)	1.0
  (647, 31)	1.0
  (647, 33)	1.0
  (647, 35)	1.0
  (647, 38)	1.0
  (647, 40)	1.0
  (647, 41)	1.0
  (648, 1)	1.0
  (648, 3)	1.0
  (648, 4)	1.0
  (648, 7)	1.0
  (648, 9)	1.0
  (648, 13)	1.0
  (648, 17)	1.0
  (648, 20)	1.0
  (648, 25)	1.0
  (648, 27)	1.0
  (648, 29)	1.0
  (648, 31)	1.0
  (648, 33)	1.0
  (648, 35)	1.0
  (648, 38)	1.0
  (648, 40)	1.0
  (648, 41)	1.0


In [18]:
cold_df = df.select_dtypes(exclude=['object'])
cold_df.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,18.0,4,4,2,2,0,4,3,4,1,1,3,4,0,11,11
1,-999.0,1,1,1,2,0,5,3,3,1,1,3,2,9,11,11
2,15.0,1,1,1,2,0,4,3,2,2,3,3,6,12,13,12
3,15.0,4,2,1,3,0,3,2,2,1,1,5,0,14,14,14
4,16.0,3,3,1,2,0,4,3,2,1,2,5,0,11,13,13


In [19]:
from scipy.sparse import csr_matrix
cold = csr_matrix(cold_df)

In [20]:
from scipy.sparse import hstack
final_sparse_matrix = hstack((hot, cold))

In [21]:
final_df = pd.DataFrame(final_sparse_matrix.toarray())
final_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,18.0,4.0,4.0,2.0,2.0,0.0,4.0,3.0,4.0,1.0,1.0,3.0,4.0,0.0,11.0,11.0
1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-999.0,1.0,1.0,1.0,2.0,0.0,5.0,3.0,3.0,1.0,1.0,3.0,2.0,9.0,11.0,11.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,15.0,1.0,1.0,1.0,2.0,0.0,4.0,3.0,2.0,2.0,3.0,3.0,6.0,12.0,13.0,12.0
3,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,15.0,4.0,2.0,1.0,3.0,0.0,3.0,2.0,2.0,1.0,1.0,5.0,0.0,14.0,14.0,14.0
4,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,16.0,3.0,3.0,1.0,2.0,0.0,4.0,3.0,2.0,1.0,2.0,5.0,0.0,11.0,13.0,13.0


In [22]:
from sklearn.base import TransformerMixin

class NullValueImputer(TransformerMixin):
  def __init__(self):
    None
  def fit(self, X, y=None):
    return self
  def transform(self, X, y=None):
    for column in X.columns.tolist():
      if column in X.columns[X.dtypes==object].tolist():
        X[column] = X[column].fillna(X[column].mode())
      else:
        X[column] = X[column].fillna(-999.0)
    return X

In [23]:
df = pd.read_csv('./student-por.csv', sep=';')
nvi = NullValueImputer().fit_transform(df)
nvi.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18.0,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,-999.0,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15.0,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15.0,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16.0,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


In [24]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

df = pd.read_csv('./student-por.csv', sep=';')

In [25]:
mode_imputer = SimpleImputer(strategy='most_frequent')
const_imputer = SimpleImputer(strategy='constant', fill_value=-999.0)

In [26]:
numeric_columns = df.columns[df.dtypes!=object].tolist()

In [27]:
ct = ColumnTransformer([('str', mode_imputer, categorical_columns),
                        ('num', const_imputer, numeric_columns)])

In [28]:
new_df = pd.DataFrame(ct.fit_transform(df),
                      columns=categorical_columns+numeric_columns)

In [29]:
new_df = new_df[df.columns]
new_df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18.0,U,GT3,A,4.0,4.0,at_home,teacher,course,mother,2.0,2.0,0.0,yes,no,no,no,yes,yes,no,no,4.0,3.0,4.0,1.0,1.0,3.0,4.0,0.0,11.0,11.0
1,GP,F,-999.0,U,GT3,T,1.0,1.0,at_home,other,course,father,1.0,2.0,0.0,no,yes,no,no,no,yes,yes,no,5.0,3.0,3.0,1.0,1.0,3.0,2.0,9.0,11.0,11.0
2,GP,F,15.0,U,LE3,T,1.0,1.0,at_home,other,other,mother,1.0,2.0,0.0,yes,no,no,no,yes,yes,yes,no,4.0,3.0,2.0,2.0,3.0,3.0,6.0,12.0,13.0,12.0
3,GP,F,15.0,U,GT3,T,4.0,2.0,health,services,home,mother,1.0,3.0,0.0,no,yes,no,yes,yes,yes,yes,yes,3.0,2.0,2.0,1.0,1.0,5.0,0.0,14.0,14.0,14.0
4,GP,F,16.0,U,GT3,T,3.0,3.0,other,other,home,father,1.0,2.0,0.0,no,yes,no,no,yes,yes,no,no,4.0,3.0,2.0,1.0,2.0,5.0,0.0,11.0,13.0,13.0


In [30]:
class SparseMatrix(TransformerMixin):
  def __init__(self):
    self.ohe = OneHotEncoder()

  def fit(self, X, y=None):
    self.categorical_columns = X.columns[X.dtypes==object].tolist()
    self.ohe.fit(X[self.categorical_columns])
    return self

  def transform(self, X, y=None):
    hot = self.ohe.transform(X[self.categorical_columns])
    cold_df = X.select_dtypes(exclude=["object"])
    cold = csr_matrix(cold_df)

    final_sparse_matrix = hstack((hot, cold))
    return final_sparse_matrix


In [31]:
sm = SparseMatrix().fit_transform(nvi)
print(sm)

  (0, 0)	1.0
  (0, 2)	1.0
  (0, 5)	1.0
  (0, 6)	1.0
  (0, 8)	1.0
  (0, 10)	1.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 25)	1.0
  (0, 28)	1.0
  (0, 29)	1.0
  (0, 31)	1.0
  (0, 33)	1.0
  (0, 36)	1.0
  (0, 38)	1.0
  (0, 39)	1.0
  (0, 41)	1.0
  (1, 0)	1.0
  (1, 2)	1.0
  (1, 5)	1.0
  (1, 6)	1.0
  (1, 9)	1.0
  (1, 10)	1.0
  (1, 17)	1.0
  (1, 20)	1.0
  :	:
  (647, 49)	2.0
  (647, 50)	4.0
  (647, 51)	5.0
  (647, 52)	3.0
  (647, 53)	4.0
  (647, 54)	2.0
  (647, 55)	6.0
  (647, 56)	10.0
  (647, 57)	10.0
  (647, 58)	10.0
  (648, 43)	18.0
  (648, 44)	3.0
  (648, 45)	2.0
  (648, 46)	3.0
  (648, 47)	1.0
  (648, 49)	4.0
  (648, 50)	4.0
  (648, 51)	1.0
  (648, 52)	3.0
  (648, 53)	4.0
  (648, 54)	5.0
  (648, 55)	4.0
  (648, 56)	10.0
  (648, 57)	11.0
  (648, 58)	11.0


In [32]:
sm_df = pd.DataFrame(sm.toarray())
sm_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,18.0,4.0,4.0,2.0,2.0,0.0,4.0,3.0,4.0,1.0,1.0,3.0,4.0,0.0,11.0,11.0
1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-999.0,1.0,1.0,1.0,2.0,0.0,5.0,3.0,3.0,1.0,1.0,3.0,2.0,9.0,11.0,11.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,15.0,1.0,1.0,1.0,2.0,0.0,4.0,3.0,2.0,2.0,3.0,3.0,6.0,12.0,13.0,12.0
3,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,15.0,4.0,2.0,1.0,3.0,0.0,3.0,2.0,2.0,1.0,1.0,5.0,0.0,14.0,14.0,14.0
4,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,16.0,3.0,3.0,1.0,2.0,0.0,4.0,3.0,2.0,1.0,2.0,5.0,0.0,11.0,13.0,13.0


In [33]:
ct2 = ColumnTransformer([('ohe', OneHotEncoder(), categorical_columns)],
                        remainder='passthrough')

In [34]:
ct2.fit(new_df)
oh_columns = ct2.named_transformers_['ohe'].get_feature_names_out()

In [35]:
import numpy as np

new_df2 = pd.DataFrame(ct2.transform(new_df),
                       columns = np.append(oh_columns, numeric_columns))
new_df2.head()

,school_GP,school_MS,sex_F,sex_M,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,schoolsup_no,schoolsup_yes,famsup_no,famsup_yes,paid_no,paid_yes,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,18.0,4.0,4.0,2.0,2.0,0.0,4.0,3.0,4.0,1.0,1.0,3.0,4.0,0.0,11.0,11.0
1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-999.0,1.0,1.0,1.0,2.0,0.0,5.0,3.0,3.0,1.0,1.0,3.0,2.0,9.0,11.0,11.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,15.0,1.0,1.0,1.0,2.0,0.0,4.0,3.0,2.0,2.0,3.0,3.0,6.0,12.0,13.0,12.0
3,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,15.0,4.0,2.0,1.0,3.0,0.0,3.0,2.0,2.0,1.0,1.0,5.0,0.0,14.0,14.0,14.0
4,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,16.0,3.0,3.0,1.0,2.0,0.0,4.0,3.0,2.0,1.0,2.0,5.0,0.0,11.0,13.0,13.0


In [36]:
df = pd.read_csv('./student-por.csv', sep=';')

In [37]:
y = df.iloc[:, -1]
X = df.iloc[:, :-3]

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [39]:
from sklearn.pipeline import Pipeline

In [40]:
data_pipeline = Pipeline([('null_imputer', NullValueImputer()),
                          ('sparse', SparseMatrix())])

In [41]:
X_train_transformed = data_pipeline.fit_transform(X_train).toarray()

In [42]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error as MSE
from xgboost import XGBRegressor

In [43]:
y_train.value_counts()

11    82
10    75
13    58
12    53
14    42
15    36
9     29
16    27
8     26
17    24
18    14
0     10
7      7
6      1
19     1
5      1
Name: G3, dtype: int64

In [44]:
kfold = KFold(n_splits=5, shuffle=True, random_state=2)

In [45]:
def cross_val(model):
  scores = cross_val_score(model, X_train_transformed, y_train,
                           scoring='neg_root_mean_squared_error', cv=kfold)
  rmse = (-scores.mean())
  return rmse

In [46]:
cross_val(XGBRegressor(missing=-999.0))

[06:38:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:38:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:38:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:38:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:38:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


2.7452680330055954

In [47]:
from sklearn.ensemble import GradientBoostingRegressor

gbr_lower = GradientBoostingRegressor(loss='quantile', alpha=0.05,
                                      random_state=2)
y_lower = gbr_lower.fit(X_train_transformed, y_train).predict(X_train_transformed)

gbr_upper = GradientBoostingRegressor(loss='quantile', alpha=0.95,
                                      random_state=2)
y_upper = gbr_upper.fit(X_train_transformed, y_train).predict(X_train_transformed)

In [48]:
np.logical_and(y_lower <= y_train,
               y_train <= y_upper).mean()

0.897119341563786

In [49]:
X_test_clean = data_pipeline.transform(X_test).toarray()

y_lower = gbr_lower.predict(X_test_clean)
y_upper = gbr_upper.predict(X_test_clean)

np.logical_and(y_lower <= y_test,
               y_test <= y_upper).mean()

0.8650306748466258

In [50]:
!pip install mapie

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
from mapie.regression import MapieRegressor

xgbr = XGBRegressor(missing=-999.0)
mapie = MapieRegressor(xgbr, cv=10, n_jobs=-1)

In [52]:
mapie.fit(X_train_transformed, y_train)

[06:38:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


MapieRegressor(cv=10, estimator=XGBRegressor(missing=-999.0), n_jobs=-1)

In [53]:
y_pred, y_pis = mapie.predict(X_test_clean, alpha=0.1)

In [54]:
from mapie.metrics import regression_coverage_score
regression_coverage_score(y_test, y_pis[:, 0], y_pis[:, 1])

0.9079754601226994

In [55]:
from mapie.subsample import Subsample

subs = Subsample(n_resamplings=30, random_state=0)

In [56]:
mapie = MapieRegressor(xgbr, cv=subs, n_jobs=-1)
mapie.fit(X_train_transformed, y_train)

[06:38:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


MapieRegressor(cv=Subsample(n_resamplings=30, n_samples=None, random_state=0, replace=True),
               estimator=XGBRegressor(missing=-999.0), n_jobs=-1)

In [57]:
y_pred, y_pis = mapie.predict(X_test_clean, alpha=0.1)
regression_coverage_score(y_test, y_pis[:, 0], y_pis[:, 1])

0.901840490797546

In [58]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train_transformed,
                                                            y_train, random_state=2)

In [59]:
def n_estimators(model):
  eval_set = [(X_test_2, y_test_2)]
  eval_metric = 'rmse'
  model.fit(X_train_2, y_train_2, eval_metric=eval_metric,
            eval_set=eval_set, early_stopping_rounds=100)
  y_pred = model.predict(X_test_2)
  rmse = MSE(y_test_2, y_pred)**0.5
  return rmse


In [60]:
n_estimators(XGBRegressor(n_estimators=5000, misssing=-999.0))

[06:41:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:10.6053
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:9.62035
[2]	validation_0-rmse:8.74792
[3]	validation_0-rmse:7.97817
[4]	validation_0-rmse:7.28649
[5]	validation_0-rmse:6.68293
[6]	validation_0-rmse:6.15832
[7]	validation_0-rmse:5.68457
[8]	validation_0-rmse:5.2865
[9]	validation_0-rmse:4.91115
[10]	validation_0-rmse:4.61207
[11]	validation_0-rmse:4.34764
[12]	validation_0-rmse:4.12113
[13]	validation_0-rmse:3.92878
[14]	validation_0-rmse:3.77603
[15]	validation_0-rmse:3.63245
[16]	validation_0-rmse:3.50531
[17]	validation_0-rmse:3.40814
[18]	validation_0-rmse:3.31759
[19]	validation_0-rmse:3.2433
[20]	validation_0-rmse:3.18486
[21]	validation_0-rmse:3.13762
[22]	validation_0-rmse:3.10825
[23]	validation_0-rmse:3.08358
[24]	validation_0-rmse:3.06351
[25]	validation_0-rmse:3.04194
[26]	validati

2.8825466337584627

In [61]:
def grid_search(params, reg=XGBRegressor(missing=-999.0)):
  grid_reg = GridSearchCV(reg, params,
                          scoring='neg_mean_squared_error', cv=kfold)
  grid_reg.fit(X_train_transformed, y_train)
  best_params = grid_reg.best_params_
  print('best params:', best_params)
  best_score = np.sqrt(-grid_reg.best_score_)
  print('best score:', best_score)

In [63]:
grid_search(params={'max_depth': [1, 2, 3, 4, 6, 7, 8],
                    'n_estimators': [85]})

[06:45:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:45:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:45:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:45:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:45:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:45:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:45:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:45:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [65]:
grid_search(params={'max_depth': [1, 2],
                    'min_child_weight': [1, 2, 3, 4, 5],
                    'n_estimators': [85]})

[06:46:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:46:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:46:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:46:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:46:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:46:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:46:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:46:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [66]:
grid_search(params={'max_depth': [2],
                    'min_child_weight': [4, 6, 7, 8, 9, 10],
                    'n_estimators': [34, 50, 85]})

[06:47:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:47:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:47:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:47:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:47:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:47:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:47:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:47:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [68]:
grid_search(params={'max_depth': [2],
                    'min_child_weight': [4, 6, 7, 8, 9, 10],
                    'subsample': [0.8, 0.9, 1],
                    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'n_estimators': [40, 50, 85]})

[06:48:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:48:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:48:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:48:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:48:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:48:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:48:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:48:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [69]:
grid_search(params={'max_depth': [2],
                    'min_child_weight': [8, 9, 10],
                    'subsample': [0.9],
                    'colsample_bytree': [1],
                    'colsample_bylevel': [0.6, 0.7, 0.8, 0.9, 1],
                    'colsample_bynode': [0.6, 0.7, 0.8, 0.9, 1],
                    'n_estimators': [85]})

[06:53:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:53:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:53:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:53:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:53:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:53:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:53:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:53:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [70]:
X_test_transformed = data_pipeline.fit_transform(X_test).toarray()

In [72]:
model = XGBRegressor(max_depth=2,
                    min_child_weight=10,
                    subsample=0.9,
                    colsample_bytree=1.0,
                    colsample_bylevel=1.0,
                    colsample_bynode=1.0,
                    n_estimators=85,
                    missing=-999.0)
model.fit(X_train_transformed, y_train)
y_pred = model.predict(X_test_transformed)
rmse = MSE(y_pred, y_test)**0.5
rmse

[06:57:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


2.8174648668447175

In [73]:
model = XGBRegressor(max_depth=2,
                    min_child_weight=13,
                    subsample=0.6,
                    colsample_bytree=1.0,
                    colsample_bylevel=1.0,
                    colsample_bynode=1.0,
                    n_estimators=85,
                    missing=-999.0)
model.fit(X_train_transformed, y_train)
y_pred = model.predict(X_test_transformed)
rmse = MSE(y_pred, y_test)**0.5
rmse

[06:58:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


2.795029457844444

In [74]:
full_pipeline = Pipeline([('null_imputer', NullValueImputer()),
                          ('sparse', SparseMatrix()),
                          ('xgb', XGBRegressor(max_depth=2,
                                               min_child_weight=13,
                                               subsample=0.6,
                                               colsample_bytree=1.0,
                                               colsample_bylevel=1.0,
                                               colsample_bynode=1.0,
                                               n_estimators=85,
                                               missing=-999.0))])

In [75]:
full_pipeline.fit(X, y)

[07:01:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(steps=[('null_imputer',
                 <__main__.NullValueImputer object at 0x7ffac01b7f50>),
                ('sparse', <__main__.SparseMatrix object at 0x7ffac01b7410>),
                ('xgb',
                 XGBRegressor(colsample_bylevel=1.0, colsample_bynode=1.0,
                              colsample_bytree=1.0, max_depth=2,
                              min_child_weight=13, missing=-999.0,
                              n_estimators=85, subsample=0.6))])

In [76]:
new_data = X_test
full_pipeline.predict(new_data)

array([13.691212 ,  8.360574 , 11.890403 , 14.047672 , 12.753544 ,
       11.87826  , 14.175683 , 11.860201 , 11.020824 , 12.709584 ,
       12.720891 ,  9.266519 , 13.207647 , 13.055632 , 13.518691 ,
        7.920157 ,  9.395828 , 10.544474 , 14.4813175, 10.456375 ,
       12.389788 , 12.909752 ,  5.0609164, 12.235758 ,  7.827029 ,
        8.04746  , 10.664764 , 13.03458  , 13.605008 , 12.048655 ,
       11.44999  , 11.821306 , 14.657296 ,  8.558382 , 11.569672 ,
       13.178446 , 12.451457 , 11.083755 ,  8.733149 , 13.117414 ,
       11.258632 , 11.274799 , 11.227485 , 14.4440775, 14.027885 ,
       14.759201 , 12.929606 , 12.893784 , 12.900142 , 12.13739  ,
       14.182906 ,  6.903615 ,  5.072094 , 12.756875 , 14.402763 ,
       10.689755 , 13.092604 ,  8.878193 , 13.421241 , 12.754645 ,
       13.677641 ,  7.5485754,  8.891133 , 12.623033 , 14.761262 ,
       11.583069 , 13.371016 , 13.81163  , 12.468255 , 10.215679 ,
       12.280749 , 12.276554 , 12.793096 ,  7.8432755, 13.2370

In [77]:
np.round(full_pipeline.predict(new_data))

array([14.,  8., 12., 14., 13., 12., 14., 12., 11., 13., 13.,  9., 13.,
       13., 14.,  8.,  9., 11., 14., 10., 12., 13.,  5., 12.,  8.,  8.,
       11., 13., 14., 12., 11., 12., 15.,  9., 12., 13., 12., 11.,  9.,
       13., 11., 11., 11., 14., 14., 15., 13., 13., 13., 12., 14.,  7.,
        5., 13., 14., 11., 13.,  9., 13., 13., 14.,  8.,  9., 13., 15.,
       12., 13., 14., 12., 10., 12., 12., 13.,  8., 13., 12., 13., 14.,
       12., 11., 13.,  7., 11., 12., 14., 11., 10., 13., 14., 15., 12.,
       10., 14., 12., 13., 14., 13.,  8., 13., 15., 13.,  9., 12., 13.,
       13., 14., 13., 11.,  9., 13.,  8., 11., 13., 11., 14., 12., 13.,
       12., 12., 12., 13.,  9., 13., 11., 14., 11., 15., 14., 11., 13.,
       10., 15.,  9., 10., 13., 14.,  9., 12., 14., 14., 10., 12., 13.,
       12., 12., 11., 13., 13., 14., 13.,  8., 11., 12.,  8.,  9., 12.,
       15., 13., 13., 10., 12., 13.,  8.], dtype=float32)

In [78]:
new_df = pd.read_csv('./student-por.csv')
new_X = df.iloc[:, :-3]
new_y = df.iloc[:, -1]
new_model = full_pipeline.fit(new_X, new_y)

[07:11:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [79]:
more_new_data = X_test[:25]
np.round(new_model.predict(more_new_data))

array([14.,  8., 12., 14., 13., 12., 14., 12., 11., 13., 13.,  9., 13.,
       13., 14.,  8.,  9., 11., 14., 10., 12., 13.,  5., 12.,  8.],
      dtype=float32)

In [90]:
model.save_model('./final_xgboost_model.json')

In [91]:
load_xgbr = XGBRegressor()
load_xgbr.load_model('./final_xgboost_model.json')
load_xgbr

[07:26:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

Not working

In [93]:
import pickle

with open('full_pipeline.pickle', 'wb') as f:
  pickle.dump(full_pipeline, f)

In [94]:
with open('full_pipeline.pickle', 'rb') as f:
  load_pipeline = pickle.load(f)

[07:28:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [95]:
np.round(load_pipeline.predict(more_new_data))

array([14.,  8., 12., 14., 13., 12., 14., 12., 11., 13., 13.,  9., 13.,
       13., 14.,  8.,  9., 11., 14., 10., 12., 13.,  5., 12.,  8.],
      dtype=float32)

In [96]:
import joblib

joblib.dump(full_pipeline, 'full_pipeline.joblib')

load_pipeline = joblib.load('full_pipeline.joblib')

np.round(load_pipeline.predict(more_new_data))

[07:30:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([14.,  8., 12., 14., 13., 12., 14., 12., 11., 13., 13.,  9., 13.,
       13., 14.,  8.,  9., 11., 14., 10., 12., 13.,  5., 12.,  8.],
      dtype=float32)